In [1]:
import math
import tensorflow as tf
import numpy as np
import matplotlib
import os
from tensorflow.contrib import rnn

tf.reset_default_graph()
tf.set_random_seed(777)

if "DISPLAY" not in os.environ:
    matplotlib.use('TkAgg')

import matplotlib.pyplot as plt

def MinMaxScaler(data):
    numerator = data - np.min(data,0)
    denominator = np.max(data,0)-np.min(data,0)
    return numerator / (denominator+1e-7)

xy = np.genfromtxt('/Users/yeseo/Desktop/City_Counted_TaxiMach_Link_Dataset_Full_201501 - 12.txt',delimiter = ',',dtype = None)
##xy_with_noise = np.genfromtxt('/Users/yeseo/Desktop/2015eliminated_1.txt',delimiter = ',',dtype = None)

#data_preprocessing
xy= xy[:,:27]
a = xy[:,:2]
b = xy[:,2:]
b= MinMaxScaler(b)
xy = np.hstack((a,b))

##xy_with_noise = xy_with_noise[:,:27]
##a_with_noise = xy_with_noise[:,:2]
##b_with_noise = xy_with_noise[:,2:]
##b_with_noise = MinMaxScaler(b_with_noise)
##xy_with_noise = np.hstack((a_with_noise,b_with_noise))


#parameters
seq_length =6
data_dim =27
hidden_dim = 54
output_dim = 25
learning_rate = 0.01
iterations = 1500

train_size = int(len(xy)*0.7)
validation_size = int(len(xy)*0.2)

#divide data set to train,validation and test set
train_set = xy[:train_size]
validation_set = xy[train_size:train_size+validation_size]
test_set = xy[train_size+validation_size:]

##train_set_with_noise = xy_with_noise[:train_size]
##validation_set_with_noise = xy_with_noise[train_size:train_size+validation_size]
##test_set_with_noise = xy_with_noise[train_size+validation_size:]

# build data set for rnn
def build_dataset(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range (0,len(time_series)-seq_length):
        _x = time_series[i:i + seq_length, :]
        _y = time_series[i+seq_length,2:]
     
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)

#train_set, test_set 만들기
trainX, trainY = build_dataset(train_set,seq_length)
validationX, validationY = build_dataset(validation_set,seq_length)
testX,testY = build_dataset(test_set, seq_length)

##trainX_with_noise, trainY_with_noise = build_dataset(train_set_with_noise,seq_length)
##validationX_with_noise, validationY_with_noise = build_dataset(validation_set_with_noise,seq_length)
##testX_with_noise,testY_with_noise = build_dataset(test_set_with_noise, seq_length)


X1 = tf.placeholder(tf.float32,[None, seq_length,data_dim])
Y1 = tf.placeholder(tf.float32,[None,25])
keep_prob = tf.placeholder(tf.float32, name='keep_prob')

##X2 = tf.placeholder(tf.float32,[None, seq_length,data_dim])
##Y2 = tf.placeholder(tf.float32,[None,25])

#LSTM CELL만들기

with tf.variable_scope("rnn1"):
    lstm_1 = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple = True, activation=tf.tanh)
    dropout1 = tf.contrib.rnn.DropoutWrapper(lstm_1,keep_prob)
    lstm_2 = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim*2, state_is_tuple = True, activation=tf.tanh)
    dropout2 = tf.contrib.rnn.DropoutWrapper(lstm_2,keep_prob)
    lstm_3 = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple = True, activation=tf.tanh)
    dropout3 = tf.contrib.rnn.DropoutWrapper(lstm_3,keep_prob)
    cell1 = tf.contrib.rnn.MultiRNNCell([dropout1,dropout2,dropout3])
    outputs1,_states1 = tf.nn.dynamic_rnn(cell1,X1,dtype = tf.float32)
    Y_pred = tf.contrib.layers.fully_connected(outputs1[:,-1], output_dim,activation_fn = None)
    loss1 =tf.reduce_sum(tf.square(Y_pred-Y1))
    train1 = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss1)

##with tf.variable_scope("rnn2"):
    ##cell2 = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple = True, activation=tf.tanh)
    ##outputs2,_states2 = tf.nn.dynamic_rnn(cell2, X2, dtype = tf.float32)
    ##Y_pred_with_noise = tf.contrib.layers.fully_connected(outputs2[:,-1], output_dim,activation_fn = None)

    ##loss2 =tf.reduce_mean(tf.square(Y_pred_with_noise-Y2))
    ##train2 = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss2)


#RMSE 측정
targets = tf.placeholder(tf.float32,[None,25])
predictions = tf.placeholder(tf.float32,[None,25])
rmse = tf.sqrt(tf.reduce_mean(tf.square(targets-predictions)))

x1 = np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25])
x2 = x1+0.3
x3 = x2+0.3
loss_for_graph = np.zeros(iterations)
x4 = np.array(range(0,iterations))
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(iterations):
        _, step_loss1 = sess.run([train1,loss1],feed_dict={X1:trainX, Y1:trainY, keep_prob : 0.7})
        print("[step: {}] loss: {}".format(i,step_loss1))
        loss_for_graph[i] = step_loss1
       ## _, step_loss2 = sess.run([train2,loss2],feed_dict={X2:trainX_with_noise, Y2:trainY_with_noise})
        ##print("[step: {}] loss: {}".format(i,step_loss2))
        
    validation_predict = sess.run(Y_pred, feed_dict = {X1:validationX, keep_prob : 1})
    ##validation_predict_with_noise = sess.run(Y_pred_with_noise, feed_dict = {X2:validationX_with_noise})
    test_predict = sess.run(Y_pred, feed_dict = {X1:testX, keep_prob : 1})
    
    rmse_val = sess.run(rmse, feed_dict={targets: validationY,predictions: validation_predict})
    print("RMSE: {}".format(rmse_val))
   # print("pred: {}".format(test_predict[-1,:]))
    #print("real: {}".format(testY[-1,:]))
    #print("noise: {}".format(eliminate_noise_pred[-1,:]))
    
#    plt.bar(x1,test_predict[-1,:],label = 'predict',color ='b',width = 0.1)
  #  plt.bar(x2,testY[-1,:],label = 'real',color ='g',width = 0.1)
    #plt.bar(x3,eliminate_noise_pred[-1,:],label = 'noise',color ='g',width = 0.1)
    plt.plot(x4,loss_for_graph)
    #plt.legend()
    plt.show()




Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
[step: 0] loss: 12110.4833984375
[step: 1] loss: 4323.2626953125
[step: 2] loss: 9444.328125
[step: 3] loss: 3525.119384765625
[step: 4] loss: 3382.76416015625
[step: 5] loss: 3057.71630859375
[step: 6] loss: 2941.58935546875
[step: 7] loss: 2655.846435546875
[step: 8] loss: 2220.332275390625
[step: 9] loss: 1879.392822265625
[step: 10] loss: 1669.26708984375
[step: 11] loss: 1581.265625
[step: 12] loss: 1586.117431640625
[step: 13] loss: 1601.7105712890625
[step: 14] loss: 1586.043212890625
[step: 15] loss: 1589.8408203125
[step: 16] loss: 1520.392578125
[step: 17] loss: 1475.1068115234375
[step: 18] loss: 1452.75927734375
[step: 19] loss: 1391.042724609375
[step: 20] loss: 1364.839111328125
[step: 21] loss: 1338.134521484375
[step: 22] loss: 1290.306274414

[step: 220] loss: 213.1904754638672
[step: 221] loss: 212.73211669921875
[step: 222] loss: 217.82501220703125
[step: 223] loss: 224.25827026367188
[step: 224] loss: 227.3611297607422
[step: 225] loss: 222.15284729003906
[step: 226] loss: 212.20867919921875
[step: 227] loss: 224.48007202148438
[step: 228] loss: 223.7059326171875
[step: 229] loss: 202.72767639160156
[step: 230] loss: 217.775634765625
[step: 231] loss: 212.9490509033203
[step: 232] loss: 209.51400756835938
[step: 233] loss: 213.2264862060547
[step: 234] loss: 213.2304229736328
[step: 235] loss: 204.075927734375
[step: 236] loss: 209.33070373535156
[step: 237] loss: 217.2452392578125
[step: 238] loss: 212.04708862304688
[step: 239] loss: 202.80963134765625
[step: 240] loss: 216.0485076904297
[step: 241] loss: 204.15435791015625
[step: 242] loss: 210.071044921875
[step: 243] loss: 225.77755737304688
[step: 244] loss: 211.52488708496094
[step: 245] loss: 202.7921905517578
[step: 246] loss: 217.59681701660156
[step: 247] loss

[step: 445] loss: 181.9324493408203
[step: 446] loss: 172.34848022460938
[step: 447] loss: 158.4947967529297
[step: 448] loss: 175.555908203125
[step: 449] loss: 175.35586547851562
[step: 450] loss: 176.8648681640625
[step: 451] loss: 173.82424926757812
[step: 452] loss: 162.99688720703125
[step: 453] loss: 177.96673583984375
[step: 454] loss: 165.3805389404297
[step: 455] loss: 166.404052734375
[step: 456] loss: 188.97506713867188
[step: 457] loss: 157.56295776367188
[step: 458] loss: 184.6947479248047
[step: 459] loss: 170.88040161132812
[step: 460] loss: 173.56024169921875
[step: 461] loss: 175.531982421875
[step: 462] loss: 168.9223175048828
[step: 463] loss: 175.78175354003906
[step: 464] loss: 163.26498413085938
[step: 465] loss: 172.740478515625
[step: 466] loss: 164.76626586914062
[step: 467] loss: 167.82351684570312
[step: 468] loss: 163.45611572265625
[step: 469] loss: 168.13851928710938
[step: 470] loss: 166.94915771484375
[step: 471] loss: 161.62399291992188
[step: 472] los

[step: 669] loss: 135.29293823242188
[step: 670] loss: 136.96336364746094
[step: 671] loss: 137.88858032226562
[step: 672] loss: 141.46804809570312
[step: 673] loss: 140.7540740966797
[step: 674] loss: 148.2294921875
[step: 675] loss: 145.49868774414062
[step: 676] loss: 137.28915405273438
[step: 677] loss: 143.3187713623047
[step: 678] loss: 155.2915496826172
[step: 679] loss: 144.67449951171875
[step: 680] loss: 136.89822387695312
[step: 681] loss: 158.85667419433594
[step: 682] loss: 146.02444458007812
[step: 683] loss: 145.28973388671875
[step: 684] loss: 155.63998413085938
[step: 685] loss: 141.19387817382812
[step: 686] loss: 150.6055908203125
[step: 687] loss: 144.66453552246094
[step: 688] loss: 139.82925415039062
[step: 689] loss: 138.3298797607422
[step: 690] loss: 138.29522705078125
[step: 691] loss: 142.5030517578125
[step: 692] loss: 135.52194213867188
[step: 693] loss: 139.72975158691406
[step: 694] loss: 140.71864318847656
[step: 695] loss: 142.36422729492188
[step: 696]

[step: 893] loss: 118.92091369628906
[step: 894] loss: 115.5360336303711
[step: 895] loss: 114.76068115234375
[step: 896] loss: 117.41609191894531
[step: 897] loss: 115.60414123535156
[step: 898] loss: 119.20248413085938
[step: 899] loss: 118.38764953613281
[step: 900] loss: 119.1675033569336
[step: 901] loss: 125.68148803710938
[step: 902] loss: 127.80007934570312
[step: 903] loss: 127.52796936035156
[step: 904] loss: 131.97088623046875
[step: 905] loss: 150.35711669921875
[step: 906] loss: 131.76507568359375
[step: 907] loss: 123.17677307128906
[step: 908] loss: 128.64564514160156
[step: 909] loss: 127.42538452148438
[step: 910] loss: 135.19442749023438
[step: 911] loss: 120.28553771972656
[step: 912] loss: 129.35423278808594
[step: 913] loss: 125.74923706054688
[step: 914] loss: 128.26365661621094
[step: 915] loss: 128.0562744140625
[step: 916] loss: 124.94207000732422
[step: 917] loss: 130.23135375976562
[step: 918] loss: 122.34615325927734
[step: 919] loss: 126.76829528808594
[ste

[step: 1113] loss: 100.78087615966797
[step: 1114] loss: 102.36605834960938
[step: 1115] loss: 107.64424133300781
[step: 1116] loss: 101.42916870117188
[step: 1117] loss: 113.10551452636719
[step: 1118] loss: 127.37864685058594
[step: 1119] loss: 109.26360321044922
[step: 1120] loss: 110.87616729736328
[step: 1121] loss: 114.26235961914062
[step: 1122] loss: 111.85792541503906
[step: 1123] loss: 114.23446655273438
[step: 1124] loss: 114.13851928710938
[step: 1125] loss: 113.09051513671875
[step: 1126] loss: 103.87846374511719
[step: 1127] loss: 104.9677734375
[step: 1128] loss: 108.00553131103516
[step: 1129] loss: 100.78653717041016
[step: 1130] loss: 105.76690673828125
[step: 1131] loss: 99.39640808105469
[step: 1132] loss: 103.08558654785156
[step: 1133] loss: 100.27616882324219
[step: 1134] loss: 102.70660400390625
[step: 1135] loss: 99.2724609375
[step: 1136] loss: 101.89788818359375
[step: 1137] loss: 102.8473892211914
[step: 1138] loss: 100.32023620605469
[step: 1139] loss: 101.

[step: 1332] loss: 93.24102020263672
[step: 1333] loss: 94.1957015991211
[step: 1334] loss: 98.72422790527344
[step: 1335] loss: 99.88102722167969
[step: 1336] loss: 93.5025634765625
[step: 1337] loss: 99.31450653076172
[step: 1338] loss: 112.08280944824219
[step: 1339] loss: 103.07140350341797
[step: 1340] loss: 94.17097473144531
[step: 1341] loss: 98.59234619140625
[step: 1342] loss: 111.58668518066406
[step: 1343] loss: 101.76875305175781
[step: 1344] loss: 91.9926986694336
[step: 1345] loss: 103.6667251586914
[step: 1346] loss: 101.00128936767578
[step: 1347] loss: 96.64269256591797
[step: 1348] loss: 98.6021728515625
[step: 1349] loss: 98.6787109375
[step: 1350] loss: 92.12442779541016
[step: 1351] loss: 100.81007385253906
[step: 1352] loss: 105.78516387939453
[step: 1353] loss: 94.54171752929688
[step: 1354] loss: 99.36588287353516
[step: 1355] loss: 105.94279479980469
[step: 1356] loss: 97.16036987304688
[step: 1357] loss: 98.78160095214844
[step: 1358] loss: 109.54652404785156
